In [1]:
import matplotlib.pyplot as plt
import numpy as np

class CSVPlot():
    def __init__(self, dataset_name, dataset_path):
        self.epsilons = [0.001, 0.0025, 0.005, 0.0075, 0.01, 0.05, 0.1, 0.5, 1]
        self.attacks = ['Fast Gradient Attack', 'Deep Fool Attack', 'FMN Attack']
        self.models = ['LeNet', 'ConvPara1', 'ConvPara2', 'ConvPara3', 'ConvLif1', 'ConvLif2']
        self.dataset_name = dataset_name

        self.model_perf = {f'{model}':[] for model in self.models}
        
        with open(dataset_path, 'r') as fh:
            for ind, line in enumerate(fh):
                self.model_perf[self.models[ind]] = [float(x) for x in line.split("\t")]

    def plotRobustnessPerEpsilon(self, attack: int):
        data = {model:[] for model in self.models}

        for i in self.models:
            model_perf = self.model_perf[i]
            match attack:
                case 0:
                    data[i] = model_perf[0::3]
                    baseline = data['LeNet']
                    np_data = np.array(data[i])
    
                    plt.plot(self.epsilons, (np.divide(baseline, np_data)-1)*100, label = f'{i}')
                    plt.title(f'Fast Gradient Attack, {self.dataset_name.upper()}')
                case 1:
                    data[i] = model_perf[1::3]
                    baseline = data['LeNet']
                    np_data = np.array(data[i])

                    plt.plot(self.epsilons, (np.divide(baseline, np_data)-1)*100, label = f'{i}')
                    plt.title(f'Deep Fool Attack, {self.dataset_name.upper()}')
                case 2:
                    data[i] = model_perf[2::3]
                    baseline = data['LeNet']
                    np_data = np.array(data[i])

                    plt.plot(self.epsilons, (np.divide(baseline, np_data)-1)*100, label = f'{i}')
                    plt.title(f'FMN Attack, {self.dataset_name.upper()}')

        plt.legend()
        plt.xlabel('Epsilon values')
        plt.ylabel('Attack Robustness Gain Percentage')
        plt.xscale('log')
        plt.show()

        return None
    
    def plotRobustnessPerLayer(self, epsilon=None):
        epsilon_ind = self.epsilons.index(epsilon)
        res_para = np.zeros((4, len(self.attacks)))
        res_lif = np.zeros((3, len(self.attacks)))
        j = k = 1

        for i in self.models:
            model_perf = self.model_perf[i]
            n = 3
            if i == 'ConvPara1' or i == 'ConvPara2' or i == 'ConvPara3':
                res_para[j] = [model_perf[i * n:(i + 1) * n] for i in range((len(model_perf) + n - 1) // n )][epsilon_ind]
                j += 1
            elif i == 'ConvLif1' or i == 'ConvLif2':
                res_lif[k] = [model_perf[i * n:(i + 1) * n] for i in range((len(model_perf) + n - 1) // n )][epsilon_ind]
                k += 1
            else:
                res_lif[0] = [model_perf[i * n:(i + 1) * n] for i in range((len(model_perf) + n - 1) // n )][epsilon_ind]
                res_para[0] = [model_perf[i * n:(i + 1) * n] for i in range((len(model_perf) + n - 1) // n )][epsilon_ind]

        res_lif = np.transpose(res_lif)
        res_para = np.transpose(res_para)

        plt.plot(range(res_para.shape[1]), res_para[0], label = 'Fast Gradient Attack: ParaLif')
        plt.plot(range(res_lif.shape[1]), res_lif[0], label = 'Fast Gradient Attack: Lif')
        plt.plot(range(res_para.shape[1]), res_para[1], label = 'Deep Fool Attack: ParaLif')
        plt.plot(range(res_lif.shape[1]), res_lif[1], label = 'Deep Fool Attack: Lif')
        plt.plot(range(res_para.shape[1]), res_para[2], label = 'FMN Attack: ParaLif')
        plt.plot(range(res_lif.shape[1]), res_lif[2], label = 'FMN Attack: Lif')
        plt.legend()
        plt.xticks([0, 1, 2, 3])
        plt.xlabel('Number of FF Layers Replaced By ParaLif / Lif')
        plt.ylabel('Attack Success Rate')
        plt.ylim((-.05, 1.05))
        plt.title(f'{self.dataset_name.upper()}, Epsilon: {epsilon}')
        plt.show()


        return None


In [6]:
dataset_name = ['mnist','fashion','kmnist','svhn']
dataset_path = ['mnist_res.txt', 'fashion_res.txt', 'kmnist_res.txt', 'svhn_res.txt']
input = 3
attack = 0
epsilon = 0.5

for i in range(3):
    test = CSVPlot('svhn', 'svhn_res.txt')
    test.plotRobustnessPerEpsilon(i)

'''for i in range(len(dataset_name)):
    test = CSVPlot(dataset_name[i], dataset_path[i])
    test.plotRobustnessPerEpsilon(attack)'''

'''test = CSVPlot(dataset_name[input], dataset_path[input])
test.plotRobustnessPerLayer(epsilon)'''


# Peters work

In [56]:
import pandas as pd
dataset_name = ['mnist','fashion','kmnist','svhn']
dataset_path = ['mnist_res.txt', 'fashion_res.txt', 'kmnist_res.txt', 'svhn_res.txt']


models = ['LeNet', 'ConvPara1', 'ConvPara2', 'ConvPara3', 'ConvLif1', 'ConvLif2']
model_perf = {f'{model}':[] for model in models}
attacks = ['FGSM', 'DeepFool', "FMN"]

results = {}

epsilons = [0.001, 0.0025, 0.005, 0.0075, 0.01, 0.05, 0.1, 0.5, 1]

datasets = []

for j, dataset in enumerate(dataset_name):
    with open(dataset_path[j], 'r') as fh:
        for i, line in enumerate(fh):
            results[models[i]] = [float(x) for x in line.split("\t")]
        data = pd.DataFrame(results)
        data = (data * 100).round(1)
        data['attack'] = pd.Series([attacks[i % 3] for i in range(9*3)])
        data['epsilon'] = pd.Series(epsilons).repeat(3).reset_index(drop=True)
        data['dataset'] = dataset.upper()
        datasets.append(data)
            


In [77]:
long_data = pd.melt(pd.concat(datasets, axis = 0), id_vars=['attack', 'epsilon', 'dataset'], value_vars=['LeNet', 'ConvPara1', 'ConvPara2', 'ConvPara3', 'ConvLif1', 'ConvLif2'], var_name='model', value_name='attack_success_rate')

by_data = long_data[long_data.dataset == 'SVHN'].drop('dataset', axis =1)
mnist_data = by_data.pivot_table(index=["attack", 'model'], columns='epsilon', values='attack_success_rate').reset_index()
mnist_data['sorting'] = pd.Series([(1 + i) % 6 for i in range(27)])
mnist_data.sort_values(['attack', 'sorting']).reset_index(drop=True).drop('sorting', axis=1)




epsilon,attack,model,0.001,0.0025,0.005,0.0075,0.01,0.05,0.1,0.5,1.0
0,DeepFool,LeNet,7.3,24.3,50.9,71.8,86.8,100.0,100.0,100.0,100.0
1,DeepFool,ConvLif1,17.9,53.7,88.5,96.1,97.2,98.1,97.6,97.7,97.8
2,DeepFool,ConvLif2,8.7,26.1,52.8,68.5,83.3,98.6,97.5,98.7,97.4
3,DeepFool,ConvPara1,8.7,24.6,41.5,52.1,56.9,85.2,90.6,96.4,94.7
4,DeepFool,ConvPara2,7.8,23.0,35.6,44.1,50.9,73.0,79.6,93.2,95.0
5,DeepFool,ConvPara3,9.4,17.6,26.9,35.9,37.2,65.1,74.5,92.7,93.5
6,FGSM,LeNet,7.5,20.5,41.0,54.4,61.5,90.5,91.1,91.5,91.0
7,FGSM,ConvLif1,15.2,42.6,65.2,72.0,76.0,87.4,88.8,92.0,92.6
8,FGSM,ConvLif2,8.1,22.4,42.0,49.5,57.4,79.5,88.1,90.7,89.9
9,FGSM,ConvPara1,9.4,21.9,42.3,53.4,61.2,87.1,91.9,94.0,93.8
